In [17]:
# @title IMPORTAÇÃO DAS BIBLIOTECAS
import pandas as pd

In [18]:
# @title FUNÇÃO PARA VERIFICAR DADOS DE DATAFRAMES
def verficarDF(data_frame):
    n_linhas, n_colunas = data_frame.shape
    print(f"Número de Linhas: {n_linhas}")
    print(f"Número de Colunas: {n_colunas}")
    print("COLUNAS:")
    for index, coluna in enumerate(data_frame.columns):
        print(f"{index} - {coluna}")

In [19]:
# @title CRIAÇÃO DO DATAFRAME E VERIFICAÇÃO DAS COLUNAS
df = pd.read_csv("recursos/datatran2022.csv",sep=";",encoding="cp1252")
verficarDF(df)

Número de Linhas: 64547
Número de Colunas: 30
COLUNAS:
0 - id
1 - data_inversa
2 - dia_semana
3 - horario
4 - uf
5 - br
6 - km
7 - municipio
8 - causa_acidente
9 - tipo_acidente
10 - classificacao_acidente
11 - fase_dia
12 - sentido_via
13 - condicao_metereologica
14 - tipo_pista
15 - tracado_via
16 - uso_solo
17 - pessoas
18 - mortos
19 - feridos_leves
20 - feridos_graves
21 - ilesos
22 - ignorados
23 - feridos
24 - veiculos
25 - latitude
26 - longitude
27 - regional
28 - delegacia
29 - uop


In [20]:
# @title REMOÇÃO DE COLUNAS NÃO UTILIZADAS
COLUNAS_SEM_USO = ["km",
                 "fase_dia",
                 "sentido_via",
                 "tipo_pista",
                 "tracado_via",
                 "uso_solo",
                 "pessoas",
                 "feridos",
                 "latitude",
                 "longitude",
                 "regional",
                 "delegacia",
                 "uop"]
df = df.drop(columns=COLUNAS_SEM_USO)
verficarDF(df)

Número de Linhas: 64547
Número de Colunas: 17
COLUNAS:
0 - id
1 - data_inversa
2 - dia_semana
3 - horario
4 - uf
5 - br
6 - municipio
7 - causa_acidente
8 - tipo_acidente
9 - classificacao_acidente
10 - condicao_metereologica
11 - mortos
12 - feridos_leves
13 - feridos_graves
14 - ilesos
15 - ignorados
16 - veiculos


In [34]:
# @title VERIFICAÇÃO DE INFORMAÇÕES NULAS OU DUPLICADAS

nulos = df.isnull().sum()
duplicados = df[df.duplicated()].sum()

n_linhas, _ = df.shape
nulos_pct = (nulos / n_linhas) * 100
duplicados_pct = (duplicados / n_linhas)  * 100

display(pd.DataFrame([nulos, nulos_pct, duplicados, duplicados_pct])
        .T
        .rename(columns={0:"Nulos", 1: "Nulos (%)", 2: "Duplicados", 3: "Duplicados (%)"})
        .round(decimals = 2)
       )

,Nulos,Nulos (%),Duplicados,Duplicados (%)
id,0.0,0.00,0.0,0.0
data_inversa,0.0,0.00,0.0,0.0
dia_semana,0.0,0.00,0.0,0.0
horario,0.0,0.00,0.0,0.0
uf,0.0,0.00,0.0,0.0
br,197.0,0.31,0.0,0.0
municipio,0.0,0.00,0.0,0.0
causa_acidente,0.0,0.00,0.0,0.0
tipo_acidente,0.0,0.00,0.0,0.0
classificacao_acidente,0.0,0.00,0.0,0.0


In [35]:
# @title MANIPULAÇÃO DA COLUNA 'BR'
df['br'] = df['br'].fillna(0)
df['br'] = df['br'].astype('int64')

In [36]:
# @title CONSTANTE USADAS NO CÓDIGO
SCHEMA = "datatran"
DICT_TABELAS_AUX = {'causa_acidente' : 'causas',
                    'tipo_acidente' : 'tipos',
                    'classificacao_acidente' : 'classificacoes',
                    'condicao_metereologica' : 'condicoes'}

In [51]:
# @title FUNÇÃO PARA GRAVAÇÃO DE ARQUIVO
def gerarArquivo(query, path):
    try:
        with open(path, "a") as arquivo:
            arquivo.write(query)
            arquivo.close()
    except Exception as e:
        print(f"Erro ao editar ou criar arquivo. {type(e)}")

In [49]:
# @title GERAÇÃO DAS TABELAS AUXILIARES 1/4
lista_tabelas = []
for coluna_df, tabela_db in DICT_TABELAS_AUX.items():
  tabela = list(set(df[coluna_df]))
  tabela.sort()
  lista_tabelas.append({tabela_db : tabela})

In [50]:
# @title GERAÇÃO DAS TABELAS AUXILIARES 2/4
def gerarDLLAux(nome_tabela, caracteres):
  tipoDado = f"VARCHAR({caracteres})"
  query = f"""CREATE TABLE {SCHEMA}.{nome_tabela} (cod SMALLINT PRIMARY KEY, descricao {tipoDado});
"""
  gerarArquivo(query,"saida/1_ddl_aux.sql")

In [40]:
# @title GERAÇÃO DAS TABELAS AUXILIARES 3/4
map_tabelas = {}
def gerarDMLAux(nome_tabela, descricoes_tabela):
  for cod, descricao in enumerate(descricoes_tabela, 1):
    query = f"""INSERT INTO {SCHEMA}.{nome_tabela} (cod, descricao) VALUES ('{cod}', '{descricao}');
"""
    gerarArquivo(query,"saida/2_dml_aux.sql")
    map_tabelas[descricao] = cod


In [52]:
# @title GERAÇÃO DAS TABELAS AUXILIARES 4/4

#verificação do tamanho máximo de caracteres
for dictTabela in lista_tabelas:
    caracteres = max([len(x) for x in dictTabela.values()]) + 1

    #chamada das funções
    for nome, descricoes in dictTabela.items():
        gerarDLLAux(nome,caracteres)
        gerarDMLAux(nome,descricoes)

In [43]:
# @title GERAÇÃO DA TABELA DE REGIÕES 1/2

#criação da tabela
query = f"""CREATE TABLE {SCHEMA}.regioes (cod SMALLINT PRIMARY KEY, regiao VARCHAR(13));
"""
gerarArquivo(query,"saida/3_ddl_regioes.sql")

In [ ]:
# @title GERAÇÃO DA TABELA DE REGIÕES 2/2

#inserção de dados
dfRegioes = pd.read_csv("recursos/regioes.csv")

for cod, regiao in zip(dfRegioes['cod'],dfRegioes['regiao']):
  query = f"""INSERT INTO {SCHEMA}.regioes (cod, regiao) VALUES ('{cod}', '{regiao}');
"""
  gerarArquivo(query,"saida/4_dml_regioes.sql")

In [31]:
# @title GERAÇÃO DA TABELA DE ESTADOS 1/2

#criação da tabela
query = f"""CREATE TABLE {SCHEMA}.estados (
    cod SMALLINT PRIMARY KEY,
    uf VARCHAR(2),
    estado VARCHAR(20),
    cod_regiao SMALLINT,
    
    FOREIGN KEY (cod_regiao) REFERENCES {SCHEMA}.regioes (cod));
"""
gerarArquivo(query,"saida/5_ddl_estados.sql")

In [14]:
# @title GERAÇÃO DA TABELA DE ESTADOS 2/2

#inserção de dados
map_estados = {}
dfEstados = pd.read_csv("recursos/estados.csv")
dados = zip(dfEstados['estado'], dfEstados['uf'], dfEstados['codigo'], dfEstados['regiao'])

for estado, uf, cod, cod_regiao in dados:
  query = f"""INSERT INTO {SCHEMA}.estados (cod, uf, estado, cod_regiao) VALUES ('{cod}','{uf}','{estado}','{cod_regiao}');
"""
  gerarArquivo(query,"saida/6_dml_estados.sql")
  map_estados[uf] = cod

,estado,uf,codigo,regiao
0,Acre,AC,12,3
1,Alagoas,AL,27,2
2,Amapá,AP,16,3
3,Amazonas,AM,13,3
4,Bahia,BA,29,2
5,Ceará,CE,23,1
6,Distrito Federal,DF,53,1
7,Espírito Santo,ES,32,4
8,Goiás,GO,52,1
9,Maranhão,MA,21,2


In [16]:
# @title GERAÇÃO DA TABELA DE MUNICIPIOS 1/3

#criação da tabela
query = f"""CREATE TABLE {SCHEMA}.municipios(
    cod SMALLINT PRIMARY KEY,
    municipio VARCHAR(33),
    cod_estado SMALLINT,
    
    FOREIGN KEY (cod_estado) REFERENCES {SCHEMA}.estados (cod));
"""
gerarArquivo(query,"saida/7_ddl_mun.sql")

In [17]:
# @title GERAÇÃO DA TABELA DE MUNICIPIOS 2/3

#manipulação do DataFrame
dfMunicipios = df[['municipio','uf']]
dfMunicipios = dfMunicipios.drop_duplicates()

dfMunicipios['uf'] = dfMunicipios['uf'].map(map_estados)
dfMunicipios

,municipio,uf
0,MARCOLANDIA,22
1,CAMPINA GRANDE DO SUL,41
2,NOVA ALVORADA DO SUL,50
3,ANGRA DOS REIS,33
4,RIBEIRAO DAS NEVES,31
...,...,...
62756,APUI,13
63409,TACARATU,26
63680,VILA RICA,51
63906,TARAUACA,12


In [18]:
# @title GERAÇÃO DA TABELA DE MUNICIPIOS 3/3

#inserção de dados
map_municipios = {}
cod = 1

for municipio, cod_estado in zip(dfMunicipios['municipio'], dfMunicipios['uf']):
  query = f"""INSERT INTO {SCHEMA}.municipios(cod,municipio,cod_estado) VALUES ('{cod}', '{municipio}', '{cod_estado}');
"""
  gerarArquivo(query,"saida/8_dml_mun.sql")
  map_municipios[municipio] = cod
  cod +=1

In [32]:
# @title GERAÇÃO DA TABELA DE DIAS DA SEMANA 1/2

#criação da tabela
query = f"""CREATE TABLE {SCHEMA}.dias_semana(cod SMALLINT PRIMARY KEY, dia_semana VARCHAR(11));
"""
gerarArquivo("saida/9_ddl_dias.sql")

In [20]:
# @title GERAÇÃO DA TABELA DE DIAS DA SEMANA 2/2

#inserção de dados
dfDias = pd.read_csv("recursos/dias_semana.csv")
map_dias = {}

for cod, dia in zip(dfDias['cod'], dfDias['dia_semana']):
  query = f"""INSERT INTO {SCHEMA}.dias_semana(cod,dia_semana) VALUES ('{cod}','{dia}');
"""
  gerarArquivo(query,"saida/10_dml_dias.sql")
  map_dias[dia] = cod

,cod,dia_semana
0,1,domingo
1,2,segunda-feira
2,3,terça-feira
3,4,quarta-feira
4,5,quinta-feira
5,6,sexta-feira
6,7,sábado


In [24]:
# @title GERAÇÃO DA TABELA DE REGISTROS DE ACIDENTES 1/2

#criação da tabela
query = f"""CREATE TABLE {SCHEMA}.acidentes (
    cod INT PRIMARY KEY,
    data_acidente DATE,
    hora TIME,
    rodovia SMALLINT,
    mortos SMALLINT,
    feridos_leves SMALLINT,
    feridos_graves SMALLINT,
    ilesos SMALLINT,
    ignorados SMALLINT,
    veiculos SMALLINT);
"""
gerarArquivo(query,"saida/11_ddl_acid.sql")

In [42]:
# @title GERAÇÃO DA TABELA DE REGISTROS DOS ACIDENTES 2/3

#inserção de dados na tabela (via COPY ou LOAD DATA)
COLUNAS_ACID = ['id',
                'data_inversa',
                'horario',
                'br',
                'mortos',
                'feridos_leves',
                'feridos_graves',
                'ilesos',
                'ignorados',
                'veiculos']

acidentes = df[COLUNAS_ACID]
acidentes.to_csv(f"saida/12_dml_acid.csv",sep=",",encoding="UTF-8",header=False,index=False)

In [ ]:
# @title GERAÇÃO DA TABELA DE REGISTROS DOS ACIDENTES 3/3

#inserção de dados
query = f"""
LOAD DATA 
INFILE 'C:/Users/aps/Documents/MS/Tratamento de Dados/saida/ocorrencias/14_ocorrencias_{i}.csv'
INTO TABLE datatran.ocorrencias
FIELDS TERMINATED BY ",";
"""
gerarArquivo(query,"saida/13_dml_acid.sql")

In [33]:
# @title GERAÇÃO DA TABELA DE OCORRÊNCIA 1/2

#criação da tabela
query = f"""CREATE TABLE {SCHEMA}.ocorrencias (
  cod_acidente INT,
  cod_dia SMALLINT,
  cod_estado SMALLINT,
  cod_municipio SMALLINT,
  cod_causa SMALLINT,
  cod_tipo SMALLINT,
  cod_classificacao SMALLINT,
  cod_condicao SMALLINT,
  
  FOREIGN KEY (cod_acidente) REFERENCES {SCHEMA}.acidentes (cod),
  FOREIGN KEY (cod_dia) REFERENCES {SCHEMA}.dias_semana (cod),
  FOREIGN KEY (cod_estado) REFERENCES {SCHEMA}.estados (cod),
  FOREIGN KEY (cod_municipio) REFERENCES {SCHEMA}.municipios (cod),
  FOREIGN KEY (cod_causa) REFERENCES {SCHEMA}.causas (cod),
  FOREIGN KEY (cod_tipo) REFERENCES {SCHEMA}.tipos (cod),
  FOREIGN KEY (cod_classificacao) REFERENCES {SCHEMA}.classificacoes (cod),
  FOREIGN KEY (cod_condicao) REFERENCES {SCHEMA}.condicoes (cod));
"""
gerarArquivo(query,"saida/14_ddl_ocorr.sql")


In [ ]:
# @title GERAÇÃO DA TABELA DE OCORRÊNCIA 2/2

COLUNAS_REG = ['id',
               'dia_semana',
               'uf',
               'municipio',
               'causa_acidente',
               'tipo_acidente',
               'classificacao_acidente',
               'condicao_metereologica']

#inserção de dados na tabela (via COPY ou LOAD DATA)
ocorrencias = df[COLUNAS_REG]
for coluna in DICT_TABELAS_AUX.keys():
  ocorrencias[coluna] = ocorrencias[coluna].map(map_tabelas)

ocorrencias['uf'] = ocorrencias['uf'].map(map_estados)
ocorrencias['municipio'] = ocorrencias['municipio'].map(map_municipios)
ocorrencias['dia_semana'] = ocorrencias['dia_semana'].map(map_dias)

ocorrencias.to_csv(f"saida/15_dml_ocorr.csv",sep=",",header=False,index=False,chunksize = 1_000)

In [101]:
data = []

estados = dfEstados['uf']

acidentes = df.groupby('uf').count()['id']

fatalidades = df.groupby('uf').sum()['mortos']

vias = []
causas = []
for uf in dfEstados['uf']:
    via = df.where(df['uf'] == uf).groupby('br').sum()['mortos']
    vias.append(via.idxmax(axis = 0))

    causa = df.where(df['uf'] == uf).groupby('causa_acidente').sum()['mortos']
    causas.append(causa.idxmax(axis = 0))



for estado, acidente, fatalidade, via, causa in zip(estados,acidentes, fatalidades, vias, causas):
  data.append({"estado" : estado,
   "acidentes" : acidente,
   "fatalidades" : fatalidade,
   "via_mais_perigosa" : via,
   "principal_causa" : causa
  })